In [1]:
%%capture
!pip install --upgrade optuna_integration

In [2]:
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import optuna.integration.lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.base import clone
from lightgbm import LGBMRegressor

SEED = 2024

In [3]:
DATA_DIR = '/kaggle/input/playground-series-s4e5'

train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
sample_sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [4]:
TARGET = 'FloodProbability'

In [5]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [6]:
features = list(test.columns)
cat_features = []

In [7]:
train['sum_all'] = train[features].sum(axis=1)
test['sum_all'] = test[features].sum(axis=1)

features = list(test.columns)

In [8]:
# LightGBM dataset
dtrain = lgb.Dataset(
    data=train[features],
    label=train[TARGET],
    feature_name=features,
    categorical_feature=cat_features)

In [9]:
base_params = {
    'objective': 'cross_entropy',
    'metric': 'cross_entropy',
    'learning_rate': 0.1,
    'boosting_type': 'gbdt',
    'force_row_wise': True,
    'verbosity': -1,
    'n_jobs': -1,
    'deterministic': True,
    'random_state': SEED
}

early_stopping = lgb.early_stopping(
    stopping_rounds=100,
    first_metric_only=True,
    verbose=False,
    min_delta=2e-4)

In [10]:
BUDGET = 60 * 60 * 7
NUM_FOLDS = 10

tuner = lgb.LightGBMTunerCV(
    time_budget=BUDGET,
    optuna_seed=SEED,
    params=base_params,
    train_set=dtrain,
    num_boost_round=10000,
    folds=KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED),
    seed=SEED,
    feature_name=features,
    categorical_feature=cat_features,
    callbacks=[early_stopping])

[I 2024-05-01 14:31:07,684] A new study created in memory with name: no-name-a5d60082-430e-4ec1-bfdd-ad8be18c7efb


In [11]:
%%time
tuner.run()

min_child_samples, val_score: 0.688587: 100%|##########| 5/5 [17:28<00:00, 209.76s/it]

CPU times: user 11h 51min 15s, sys: 12min 59s, total: 12h 4min 14s
Wall time: 3h 34min 55s


In [12]:
def comp_metric(y_true, y_pred):
    return r2_score(y_true, y_pred)

def custom_cv(estimator, seed=SEED, verbose=True):
    X_test = test[features]
    
    oof_preds, test_preds = {}, {}
    scores = []

    cv = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=seed)
    for fold, (train_ids, val_ids) in enumerate(cv.split(train)):
        X_train, y_train = train[features].iloc[train_ids], train[TARGET].iloc[train_ids]
        X_val, y_val = train[features].iloc[val_ids], train[TARGET].iloc[val_ids]
        
        model = clone(estimator)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[early_stopping])

        val_preds = model.predict(X_val)
        oof_preds.update(dict(zip(val_ids, val_preds)))
        test_preds[f'fold{fold}'] = model.predict(X_test)

        score = comp_metric(y_val, val_preds)
        scores.append(score)
        if verbose:
            print(f'Fold #{fold:>2}: {score:.5f} ({model.best_iteration_:>4} rounds)')
        _ = gc.collect()

    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mean'] = test_preds.mean(axis=1) # mean of fold-wise predictions
    
    oof_preds = pd.Series(oof_preds).sort_index()
    print(f'\nAvg score: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    print(f'OOF score: {comp_metric(train[TARGET], oof_preds):.5f}\n')
    
    return oof_preds, test_preds

In [13]:
%%time
model = LGBMRegressor(**tuner.best_params, n_estimators=10000)
op, tp = custom_cv(model)

Fold # 0: 0.86345 (  39 rounds)
Fold # 1: 0.86355 (  39 rounds)
Fold # 2: 0.86386 (  39 rounds)
Fold # 3: 0.86491 (  39 rounds)
Fold # 4: 0.86479 (  39 rounds)
Fold # 5: 0.86286 (  39 rounds)
Fold # 6: 0.86347 (  38 rounds)
Fold # 7: 0.86342 (  40 rounds)
Fold # 8: 0.86420 (  39 rounds)
Fold # 9: 0.86316 (  40 rounds)

Avg score: 0.86377 +/- 0.00064
OOF score: 0.86377

CPU times: user 10min 25s, sys: 2.34 s, total: 10min 27s
Wall time: 10min 27s


In [14]:
def create_submission_files(preds, notebook='01'):
    for col in preds.columns:
        sub = sample_sub.copy()
        sub[TARGET] = preds[col]
        sub.to_csv(f'nb{notebook}_{col}.csv', index=False)

In [15]:
create_submission_files(tp)

In [16]:
!head nb01_mean.csv

id,FloodProbability
1117957,0.5746264159936313
1117958,0.45368198914229296
1117959,0.45462980121386387
1117960,0.47165973133638756
1117961,0.4739681589684575
1117962,0.5083381917599572
1117963,0.5393328919894695
1117964,0.5270388432355311
1117965,0.4750023364876803
